In [1]:
!pip install fredapi

In [3]:
import os
from datetime import datetime

In [4]:
SECRET_PATH = '../secrets'
DATA_PATH = '../data'

In [17]:
from fredapi import Fred
import pandas as pd

# Initialize the FRED API client with your API key
fred = Fred(api_key=fred_api_key)

# Define the FRED series IDs for the data you want to download
series_ids = [
    'UNRATE',  # Unemployment rate
    'GDP',     # Gross domestic product
    'CPALTT01USM657N',  # Consumer Price Index for All Urban Consumers: All Items
]

# Download the FRED data for the specified series IDs
data = {}
for series_id in series_ids:
    series = fred.get_series(series_id)
    data[series_id] = series

# Combine the data into a single dataframe and save it to a CSV file
df = pd.DataFrame(data)

try:
    df.to_csv(os.path.join(DATA_PATH, formatted_date_time, 'fred_data.csv'), index=False)
except FileNotFoundError as e:
    os.mkdir(os.path.join(DATA_PATH, formatted_date_time))
    df.to_csv(os.path.join(DATA_PATH, formatted_date_time, 'fred_data.csv'), index=False)


In [20]:
import os
from datetime import datetime, timedelta
import pandas as pd
from fredapi import Fred


def get_fred_data(api_key, series_ids):
    csv_filename = 'fred_data.csv'
    csv_filename_path = os.path.join(DATA_PATH, csv_filename)

    # Check if cached data exists and is less than a month old
    if os.path.exists(csv_filename):
        file_age = datetime.now() - datetime.fromtimestamp(os.path.getmtime(csv_filename))
        if file_age < timedelta(days=30):
            # Read data from CSV file
            df = pd.read_csv(csv_filename)
            return df[df['series_id'].isin(series_ids)]

    # Read data from FRED API
    fred = Fred(api_key=api_key)
    df = pd.DataFrame()
    for series_id in series_ids:
        data = fred.get_series(series_id)
        df = df.append(pd.DataFrame({'series_id': [series_id] * len(data), 'date': data.index, 'value': data.values}))
    try:
        df.to_csv(csv_filename_path, index=False)
    except FileNotFoundError as e:
        os.mkdir(csv_filename)
        df.to_csv(csv_filename_path, index=False)

    return df

In [29]:
def main():

    with open(os.path.join(SECRET_PATH,'fred_api_key.txt')) as f:
        fred_api_key = f.read()

    series_ids_df = pd.read_csv('../metadata/fred_series_ids.txt')
    
    fred_df = get_fred_data(fred_api_key, series_ids)
    
    return fred_df

fred_df = main()

In [31]:
fred_df.pivot(index='date', columns = 'series_id')

value                  
series_id  CPALTT01USM657N        GDP UNRATE
date                                        
1946-01-01             NaN        NaN    NaN
1946-04-01             NaN        NaN    NaN
1946-07-01             NaN        NaN    NaN
1946-10-01             NaN        NaN    NaN
1947-01-01             NaN    243.164    NaN
...                    ...        ...    ...
2022-10-01        0.405649  26144.956    3.7
2022-11-01       -0.101003        NaN    3.6
2022-12-01       -0.307009        NaN    3.5
2023-01-01        0.799536        NaN    3.4
2023-02-01             NaN        NaN    3.6

[910 rows x 3 columns]